In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT,GPSConfig,ResultConfig
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import mapping
import json
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

In [2]:
#load network data and graph 
network = Network("../osmnx_example/beijing/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

[2025-02-24 12:04:54.532] [info] [network.cpp:72] Read network from file ../osmnx_example/beijing/edges.shp
Nodes 80218 edges 199609
[2025-02-24 12:04:55.141] [info] [network.cpp:172] Number of edges 199609 nodes 80218
[2025-02-24 12:04:55.141] [info] [network.cpp:173] Field index: id 18 source 0 target 1
[2025-02-24 12:04:55.242] [info] [network.cpp:176] Read network done.
[2025-02-24 12:04:55.243] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-02-24 12:04:55.257] [info] [network_graph.cpp:30] Graph nodes 80218 edges 199609
[2025-02-24 12:04:55.257] [info] [network_graph.cpp:31] Construct graph from network edges end


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [3]:
#from fmm import UBODTGenAlgorithm
#ubodt_gen = UBODTGenAlgorithm(network,graph)
#status = ubodt_gen.generate_ubodt("../osmnx_example/beijing/ubodt.txt", 4, binary=False, use_omp=True)
#print(status)

### FMM model creation and map matching

match single trajectory

In [ ]:
df = pd.read_csv("../osmnx_example/beijing/trajectorieONE.csv", sep=";")

#trajectory 1
wkt = df.loc[df["id"] == 1, "geom"].values[0]

print(wkt)

ubodt = UBODT.read_ubodt_csv("../osmnx_example/beijing/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

# configuration parameters
k = 50
radius = 20
gps_error = 10
fmm_config = FastMapMatchConfig(k,radius,gps_error)

result = model.match_wkt(wkt,fmm_config)
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

LINESTRING(116.319236 39.984094, 116.319322 39.984198, 116.319402 39.984224, 116.319389 39.984211, 116.319422 39.984217, 116.319865 39.98471, 116.31981 39.984674, 116.319773 39.984623, 116.319732 39.984606, 116.319728 39.984555, 116.319769 39.984579, 116.319769 39.984579, 116.319766 39.984577, 116.319822 39.984611, 116.319969 39.984959, 116.320056 39.985036, 116.320037 39.984741, 116.32012 39.98462, 116.320242 39.98453, 116.320331 39.984508, 116.320443 39.984537, 116.320573 39.984529, 116.320683 39.984466, 116.320778 39.984409, 116.320808 39.98432, 116.320826 39.984252, 116.320844 39.984238, 116.320853 39.984232, 116.32087 39.984246, 116.320876 39.984266, 116.320922 39.984284, 116.320935 39.984349, 116.320924 39.98436, 116.320914 39.984334, 116.320923 39.984225, 116.320926 39.984119, 116.320958 39.984007, 116.32098 39.983916, 116.321003 39.98382, 116.321005 39.983724, 116.321032 39.98363, 116.321053 39.983535, 116.321077 39.983434, 116.321087 39.983329, 116.321099 39.983212, 116.321126

In [6]:
import folium
from shapely.wkt import loads

original_geom = loads(wkt)
matched_geom = loads(result.mgeom.export_wkt())

m = folium.Map(location=[original_geom.coords[0][1], original_geom.coords[0][0]], zoom_start=16)

folium.PolyLine([(lat, lon) for lon, lat in original_geom.coords], color="red", weight=3, opacity=0.8, tooltip="Original GPS Trajectory").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in matched_geom.coords], color="blue", weight=3, opacity=0.8, tooltip="Matched Path").add_to(m)

for lon, lat in original_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red").add_to(m)
for lon, lat in matched_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="blue", fill=True, fill_color="blue").add_to(m)

# Display map 
m

### Map Matching after obfuscation